In [1]:
import glob
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import functions as fun
sourcedir = '/Volumes/My Passport/cmip5/cmip5'

In [2]:
def Amasksmb(model):
    print('smb',model)
    with xr.open_dataset(f'{sourcedir}/fx/sftlf/{model}/r0i0p0/sftlf_fx_{model}_historical_r0i0p0.nc') as ds:
        lon = ds['lon'].values
        lat = ds['lat'].values
        sft = ds['sftlf'].values    

    lon[lon<0] = lon[lon<0]+360
    lons,lats = np.meshgrid(lon,lat)
    mask = np.ones(sft.shape)
    mask[lats>-60] = 0
    mask = mask*sft/np.max(sft)

    mask2 = xr.DataArray(mask,dims=('lat','lon'),coords={'lat':lat,'lon':lon})

    ds = xr.Dataset({'mask':mask2})    
    ds.to_netcdf(f'data/Amasksmb/{model}.nc')
    ds.close()

    fig,ax = plt.subplots(1,1)
    ax.pcolor(lons,lats,mask,cmap=plt.get_cmap('Blues_r'))
    plt.savefig(f'figures/Amasksmb/{model}.png')
    plt.close()

In [3]:
def Amaskdyn(model):
    print('dyn',model)
    with xr.open_dataset(f'{sourcedir}/fx/sftof/{model}/r0i0p0/sftof_fx_{model}_historical_r0i0p0.nc') as ds:
        lon = ds['lon'].values
        lat = ds['lat'].values
        sft = ds['sftof'].values 
    
    lon[lon<0] = lon[lon<0]+360
    
    if len(lon.shape)==1:
        lon,lat = np.meshgrid(lon,lat)       
        
    with xr.open_dataset(f'{sourcedir}/fx/deptho/{model}/r0i0p0/deptho_fx_{model}_historical_r0i0p0.nc') as ds:
        dep = ds['deptho'].values
        
    files = glob.glob(f'{sourcedir}/{model}/r1i1p1/thetao_Omon_{model}_rcp85*')
    with xr.open_dataset(files[0]) as ds:
        lev = ds['lev_bnds'].values
    
    lons = np.repeat(lon[np.newaxis,:,:],len(lev),axis=0)
    lats = np.repeat(lat[np.newaxis,:,:],len(lev),axis=0)

    mask = np.repeat(np.zeros(lons.shape)[np.newaxis,:,:,:],4,axis=0)
    basin = np.arange(0,4)
    levs = (lev[:,0]+lev[:,1])/2.

    if model == 'inmcm4':
        lev2 = np.repeat(lev[:,:,np.newaxis],dep.shape[0],axis=2)
        lev2 = np.repeat(lev2[:,:,:,np.newaxis],dep.shape[1],axis=3)
        dep2 = np.repeat(dep[np.newaxis,:,:],lev.shape[0],axis=0)
        dep2 = np.repeat(dep2[:,np.newaxis,:,:],lev.shape[1],axis=1)
        lev3 = -dep2*lev2
    
    for b in range(0,4):
        mm = np.zeros(lons.shape)
        if b==0:
            #EAIS
            mm[((lons<173) | (lons>350)) & (lats<-65) & (lats>-76)] = 1
            depp = 369
        if b==1:
            #WEDD
            mm[(lons>295) & (lons<350) & (lats<-72)] = 1
            depp = 420
        if b==2:
            #AMUN
            mm[(lons>210) & (lons<295) & (lats<-70)] = 1
            depp = 305
        if b==3:
            #ROSS
            mm[((lons>150) & (lons<210)) & (lats<-76)] = 1
            depp = 312

        if model=='inmcm4':
            mm[:,dep<depp-100.] = 0
            for j in range(0,lon.shape[0]):
                for i in range(0,lon.shape[1]):
                    if mm[-1,j,i] == 0: continue
                    i0 = np.argmax(lev3[:,1,j,i]>depp-100.)
                    mm[:i0,j,i] = 0
                    w0 = (lev3[i0,1,j,i]-(depp-100.))/(lev3[i0,1,j,i]-lev3[i0,0,j,i])
                    mm[i0,j,i] = w0*mm[i0,j,i]
                    z1 = min(dep[j,i],depp+100.)
                    i1 = np.argmin(lev3[:,1,j,i]<z1)
                    w1 = (z1-lev3[i1,0,j,i])/(lev3[i1,1,j,i]-lev3[i1,0,j,i])
                    mm[i1,j,i] = w1*mm[i1,j,i]
                    mm[i1+1:,j,i] = 0
        else:
            mm[:,dep<depp-100.] = 0
            i0 = np.argmax(lev[:,1]>depp-100.)
            mm[:i0,:,:] = 0
            w0 = (lev[i0,1]-(depp-100.))/(lev[i0,1]-lev[i0,0])
            mm[i0,:,:] = w0*mm[i0,:,:]
            for j in range(0,lon.shape[0]):
                for i in range(0,lon.shape[1]):
                    if mm[-1,j,i]==0: continue
                    z1 = min(dep[j,i],depp+100.)
                    i1 = np.argmin(lev[:,1]<z1)
                    w1 = (z1-lev[i1,0])/(lev[i1,1]-lev[i1,0])
                    mm[i1,j,i] = w1*mm[i1,j,i]
                    mm[i1+1:,j,i] = 0
        mask[b,:,:,:] = mm    
    
    #Save
    mask2 = xr.DataArray(mask,dims=('basin','lev','y','x'),coords={'basin':basin,'lev':levs,'y':np.arange(lat.shape[0]),'x':np.arange(lat.shape[1])})
    lat2  = xr.DataArray(lat,dims=('y','x'),coords={'y':np.arange(lat.shape[0]),'x':np.arange(lat.shape[1])})
    lon2  = xr.DataArray(lon,dims=('y','x'),coords={'y':np.arange(lon.shape[0]),'x':np.arange(lon.shape[1])})
    
    ds = xr.Dataset({'mask':mask2,'lat':lat2,'lon':lon2})
    ds.to_netcdf(f'data/Amaskdyn/{model}.nc')
    ds.close()
    

In [4]:
mods = fun.models()
for model in mods:
#    Amasksmb(model)
    Amaskdyn(model)

dyn bcc-csm1-1


/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in less


dyn CanESM2
dyn CCSM4
dyn CNRM-CM5


/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in less


dyn CSIRO-Mk3-6-0
dyn GFDL-CM3


...

/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in less


dyn IPSL-CM5A-LR
dyn MIROC-ESM
dyn MIROC5


/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in less


dyn MPI-ESM-LR
dyn MRI-CGCM3
dyn NorESM1-M


/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in less
